Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.

SPDX-License-Identifier: Apache-2.0


# Notebook for data split and transform for time series model 
## This notebook consists of steps to 
1. process raw data into independent time series for time series model 
2. train/test data split 

In [1]:
import pandas as pd
import numpy as np

# Load raw data

In [2]:
raw_data_path = '../../data/01_raw/financial_fraud/bs140513_032310.csv'

In [3]:
raw_trans_data = pd.read_csv(raw_data_path)

In [4]:
raw_trans_data.shape

(594643, 10)

In [5]:
raw_trans_data.head(10)

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud
0,0,'C1093826151','4','M','28007','M348934600','28007','es_transportation',4.55,0
1,0,'C352968107','2','M','28007','M348934600','28007','es_transportation',39.68,0
2,0,'C2054744914','4','F','28007','M1823072687','28007','es_transportation',26.89,0
3,0,'C1760612790','3','M','28007','M348934600','28007','es_transportation',17.25,0
4,0,'C757503768','5','M','28007','M348934600','28007','es_transportation',35.72,0
5,0,'C1315400589','3','F','28007','M348934600','28007','es_transportation',25.81,0
6,0,'C765155274','1','F','28007','M348934600','28007','es_transportation',9.10,0
7,0,'C202531238','4','F','28007','M348934600','28007','es_transportation',21.17,0
8,0,'C105845174','3','M','28007','M348934600','28007','es_transportation',32.40,0
9,0,'C39858251','5','F','28007','M348934600','28007','es_transportation',35.40,0


# Save (customer_id, category) that had more than 10 entries into time series 

In [6]:
raw_trans_data.shape

(594643, 10)

In [7]:
customer_category_trans_count = raw_trans_data.groupby(by=['customer','category']).agg({'step':'count'})
customer_category_trans_count

step
customer      category                     
'C1000148617' 'es_food'                   5
              'es_health'                 2
              'es_home'                   1
              'es_hotelservices'          1
              'es_hyper'                  1
              'es_otherservices'          2
              'es_sportsandtoys'          1
              'es_tech'                   2
              'es_transportation'       114
              'es_wellnessandbeauty'      2
'C100045114'  'es_barsandrestaurants'     2
              'es_fashion'                1
              'es_food'                   4
              'es_health'                10
              'es_home'                   2
              'es_hotelservices'          2
              'es_hyper'                  2
              'es_sportsandtoys'          3
              'es_transportation'        65
              'es_travel'                 1
              'es_wellnessandbeauty'     17
'C1000699316' 'es_food'                   6
              'es_health'                 1
              'es_hotelservices'          1
              'es_hyper'                  1
              'es_otherservices'          1
              'es_transportation'        80
              'es_wellnessandbeauty'      4
'C1001065306' 'es_fashion'                1
              'es_health'                16
...                                     ...
'C998690782'  'es_contents'               1
              'es_fashion'                1
              'es_food'                   2
              'es_health'                 1
              'es_hyper'                  2
              'es_tech'                   1
              'es_transportation'       155
              'es_travel'                 2
              'es_wellnessandbeauty'      1
'C998987490'  'es_contents'               1
              'es_fashion'                1
              'es_food'                  13
              'es_health'                11
              'es_transportation'       150
              'es_wellnessandbeauty'      1
'C999393223'  'es_barsandrestaurants'     1
              'es_food'                   6
              'es_health'                 4
              'es_home'                   1
              'es_hyper'                  4
              'es_sportsandtoys'          1
              'es_transportation'       109
              'es_wellnessandbeauty'     16
'C999723254'  'es_fashion'                2
              'es_food'                   8
              'es_health'                 4
              'es_hyper'                  2
              'es_otherservices'          1
              'es_transportation'       104
              'es_wellnessandbeauty'      1

[31844 rows x 1 columns]

In [8]:
ts_count_threshold = 10

In [9]:
customer_category_trans_more_than_threshold =customer_category_trans_count.loc[customer_category_trans_count.step>ts_count_threshold].reset_index()

In [10]:
customer_category_trans_more_than_threshold

,customer,category,step
0,'C1000148617','es_transportation',114
1,'C100045114','es_transportation',65
2,'C100045114','es_wellnessandbeauty',17
3,'C1000699316','es_transportation',80
4,'C1001065306','es_health',16
5,'C1002658784','es_transportation',108
6,'C1002759277','es_transportation',147
7,'C1004109477','es_transportation',140
8,'C1004300450','es_transportation',152
9,'C1004300450','es_wellnessandbeauty',12


In [11]:
customer_category_pairs_for_ts = np.array(customer_category_trans_more_than_threshold[['customer','category']])

In [12]:
customer_category_pairs_for_ts, customer_category_pairs_for_ts.shape

(array([["'C1000148617'", "'es_transportation'"],
        ["'C100045114'", "'es_transportation'"],
        ["'C100045114'", "'es_wellnessandbeauty'"],
        ...,
        ["'C999393223'", "'es_transportation'"],
        ["'C999393223'", "'es_wellnessandbeauty'"],
        ["'C999723254'", "'es_transportation'"]], dtype=object),
 (5216, 2))

In [13]:
i = 0
customer_category_pairs_example = []
for c_m_p in customer_category_pairs_for_ts:
    if i>1: #only see two examples
        break
    c_m_p_data = raw_trans_data.loc[(raw_trans_data.customer==c_m_p[0])&( raw_trans_data.category==c_m_p[1])][['step','amount','fraud']]
    #print(len(c_m_p_data))
    if np.sum(c_m_p_data['fraud'])>0 and len(c_m_p_data)>10:
        print(c_m_p, c_m_p_data)
        customer_category_pairs_example.append(c_m_p)
        i+=1


["'C100045114'" "'es_wellnessandbeauty'"]         step  amount  fraud
18554      7   33.98      0
190340    65  187.85      0
202388    69  130.24      0
329377   107   35.37      0
333710   108  137.28      0
333711   108   30.99      0
333712   108   87.45      0
358842   115  152.87      0
358843   115   93.60      0
358844   115    5.30      0
452715   141    5.53      0
452716   141   56.50      0
452717   141   88.57      0
452718   141  185.77      0
507112   156   30.31      0
510372   157   48.90      1
554679   169   61.93      0
["'C1001065306'" "'es_health'"]         step   amount  fraud
56918     21    37.10      0
84525     31   108.32      0
84526     31   188.94      0
100311    36   906.87      1
106319    38   146.25      0
181718    63    31.41      0
181719    63   177.82      0
181720    63   106.47      0
383480   122  1024.36      1
400723   127    80.72      1
406796   128    26.65      0
406797   128    65.77      0
423608   133   185.40      0
423609   133   1

In [14]:
customer_category_pairs_example

[array(["'C100045114'", "'es_wellnessandbeauty'"], dtype=object),
 array(["'C1001065306'", "'es_health'"], dtype=object)]

# fetch the time series for the selcted pairs


In [15]:
example_c = """'C1001065306'"""
example_m = """'es_health'"""

In [16]:
c_m_p_data_example = raw_trans_data.loc[
    (raw_trans_data.customer==example_c)
    &( raw_trans_data.category==example_m)
][['step','amount','fraud']]

In [17]:
c_m_p_data_example

,step,amount,fraud
56918,21,37.10,0
84525,31,108.32,0
84526,31,188.94,0
100311,36,906.87,1
106319,38,146.25,0
181718,63,31.41,0
181719,63,177.82,0
181720,63,106.47,0
383480,122,1024.36,1
400723,127,80.72,1


In [18]:
example_ts_file_path = f"""../../data/02_intermediate/financial_fraud/ts_data/{example_c}_{example_m}_transaction_data.csv"""

In [19]:
example_ts_file_path

"../../data/02_intermediate/financial_fraud/ts_data/'C1001065306'_'es_health'_transaction_data.csv"

In [20]:
import sys
sys.path.append('../../src/')

In [21]:
from anomaly_detection_spatial_temporal_data.utils import ensure_directory

In [22]:
ensure_directory(example_ts_file_path)

In [23]:
c_m_p_data_example.rename(columns={'step':'timestamp','amount':'value','fraud':'label'}, inplace=True)
c_m_p_data_example[['timestamp','value']].to_csv(example_ts_file_path, index=False)

In [24]:
example_ts_label_file_path = f"""../../data/02_intermediate/financial_fraud/ts_label/{example_c}_{example_m}_transaction_label.csv"""

In [25]:
ensure_directory(example_ts_label_file_path)

In [26]:
c_m_p_data_example[['label']].to_csv(example_ts_label_file_path, index=False)

### generate a dummy label dict needed for NAB model

In [27]:
from pathlib import Path
import json

def generate_dummy_labels(data_dir: str, label_dir:str) -> str:
    """Generate a dummy label JSON file and return its path"""
    data_dir_path = Path(data_dir)
    dummy_labels = dict()
    for file_path in data_dir_path.rglob("*.csv"):
        file_path_relative = file_path.relative_to(data_dir_path)
        dummy_labels[str(file_path_relative)] = []
    dummy_label_path = Path(f"{label_dir}/labels-combined.json")
    with dummy_label_path.open("w") as file:
        json.dump(dummy_labels, file, indent=4)
    return str(dummy_label_path.resolve())

In [28]:
label_dict_filepath = generate_dummy_labels(
    "../../data/02_intermediate/financial_fraud/ts_data/",
    "../../data/02_intermediate/financial_fraud/ts_label/"
)

In [29]:
label_dict_filepath

'/home/ec2-user/SageMaker/anomaly-detection-spatial-temporal-data-workshop/data/02_intermediate/financial_fraud/ts_label/labels-combined.json'

# References

Edgar Alonso Lopez-Rojas and Stefan Axelsson. 2014. BANKSIM: A BANK PAYMENTS SIMULATOR FOR FRAUD DETECTION RESEARCH.

Alexander Lavin and Subutai Ahmad. 2015. Evaluating Real-Time Anomaly Detection Algorithms – The Numenta Anomaly Benchmark.